# 딥러닝모델과추론 기말프로젝트

- 딥러닝모델과 추론
- 데이터과학융합스쿨 20173237 심창우

In [105]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [106]:
data = pd.read_csv("C:/Users/lkisa/Downloads/HN16_ALL.csv")

############################################################## 회귀모델
# 허리둘레 - BMI 데이터 2차원 배열로 변환
HE_BMI = data[["HE_wc", "HE_BMI"]]
HE_BMI = HE_BMI.replace('', -1)
HE_BMI = HE_BMI.replace(' ', -1)
HE_BMI = HE_BMI.astype(float)
HE_BMI = HE_BMI[data.HE_wc != -1]
HE_BMI = HE_BMI[data.HE_BMI != -1]

HE_wc = np.array(HE_BMI["HE_wc"].astype(float))
HE_BMI = np.array(HE_BMI["HE_BMI"].astype(float))

HE_wc = HE_wc.reshape(len(HE_wc), 1)
HE_BMI = HE_BMI.reshape(len(HE_BMI), 1)

# 허리둘레 - BMI정규화
mm_scaler_HE_BMI = preprocessing.MinMaxScaler()
HE_wc = mm_scaler_HE_BMI.fit_transform(HE_wc)
HE_BMI = mm_scaler_HE_BMI.transform(HE_BMI)


############################################################## 분류모델
# 데이터 전처리
HE_HCHOL = data[["age",  "HE_chol", "HE_HCHOL"]]
HE_HCHOL = HE_HCHOL.replace('', -1)
HE_HCHOL = HE_HCHOL.replace(' ', -1)
HE_HCHOL = HE_HCHOL.astype(float)
HE_HCHOL = HE_HCHOL[data.age != -1]
HE_HCHOL = HE_HCHOL[data.HE_chol != -1]
HE_HCHOL = HE_HCHOL[HE_HCHOL.age != -1]

# 19세 이상만 필터링
data = HE_HCHOL[HE_HCHOL['age'] >= 19]

# float, int 변경
HE_chol = np.array(data["HE_chol"].astype(float))
HE_HCHOL = np.array(data["HE_HCHOL"].astype(int))

HE_chol = HE_chol.reshape(len(HE_chol), 1)
HE_HCHOL = HE_HCHOL.reshape(len(HE_HCHOL), 1)

# BMI - 고콜레스테롤혈증 유무 졍규화. 콜레스테롤 유무는 1, 0값이기 때문에 하지 않음
mm_scaler_fat = preprocessing.MinMaxScaler()
HE_chol = mm_scaler_fat.fit_transform(HE_chol)



In [107]:
# 허리둘레 - BMI 모델
model_BMI = Sequential()
model_BMI.add(Dense(1, input_dim=1))

model_BMI.summary()


# 고콜레스테롤혈증 
model_fat = Sequential()

# relu -> 0 또는 1이기 때문에 activation sigmoid 사용
model_fat.add(Dense(1, input_dim=1, activation='relu'))
model_fat.add(Dense(1, activation='sigmoid'))

model_fat.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 1)                 2         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 2         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [108]:
# 허리둘레 - BMI 모델 컴파일
model_BMI.compile(loss='mse', optimizer='sgd') 

# 고콜레스테롤혈증 - 고콜레스테롤혈증 모델 컴파일. 결과값이 0 또는 1 즉 이진분류이기 때문에 binary_crossentropy 사용함
model_fat.compile(loss='binary_crossentropy', optimizer='sgd') 

In [109]:
# 허리둘레 - BMI 모델 적합 (model fitting)
fit = model_BMI.fit(HE_wc, HE_BMI, epochs=30, batch_size=20, verbose=False)


# 고콜레스테롤혈증 - 고콜레스테롤혈증 모델 적합 (model fitting)
fit = model_fat.fit(HE_chol, HE_HCHOL, epochs=30, batch_size=20, verbose=False)


In [111]:
# 허리둘레 입력
input_wc = 80
input_mm_BMI = mm_scaler_HE_BMI.transform(np.array([input_wc]).reshape(-1, 1))

BMI_predict = model_BMI.predict(input_mm_BMI)
BMI_predict = mm_scaler_HE_BMI.inverse_transform(BMI_predict)
print("1번)", "허리둘레가", input_wc, " 일 때 예측 BMI는 약", round(BMI_predict[0][0]), "이다")

# 콜레스테롤 입력 
input_chol = 100

input_mm_fat = mm_scaler_fat.transform(np.array([input_chol]).reshape(-1, 1))
fat_predict = model_fat.predict(input_mm_fat)
print("2번)", "콜레스테롤 수치가", input_chol, "일 때 고콜레스테롤혈증에 걸릴 확률은", round(fat_predict[0][0]*100), "% 이다")

1번) 허리둘레가 80  일 때 예측 BMI는 약 23 이다
2번) 콜레스테롤 수치가 100 일 때 고콜레스테롤혈증에 걸릴 확률은 8 % 이다


In [ ]:
결론 : 이번 과제를 계기로 다양한 변수들을 다루어 보면서, 상관관계가 있는지 없는지 
    직접 하나하나 대입해보며 재미를 느꼈다. R만을 배웠지만 파이썬을 배우게 되어 매우 유익한 강의였다.
    직접 변수를 골라서 작업하는 과정은 흥미로웠고, 미래를 꿈꿀 수 있는 시간이었다.